In [1]:
import sys
sys.path.append('/Users/huibmeulenbelt/PycharmProjects/ufc')
sys.path.append('/Users/huibmeulenbelt/PycharmProjects/ufc/scripts')
sys.path.append('/Users/huibmeulenbelt/PycharmProjects/ufc/scripts/markov')

In [2]:
import json
import numpy as np
from networkx.readwrite import json_graph
from fight import Fight

In [3]:
PATH_G = '/Users/huibmeulenbelt/PycharmProjects/ufc/scripts/markov/tests/12120.json'
G = json_graph.adjacency_graph(json.load(open(PATH_G)))

Let $r$ represent all the transient states and $s$ represents the absorbing states. The goal is to compute the probability of being not absorbed after $n$ steps.

$$
P(\text{not absorbed after n steps}) = \sum_{i \in r}P(X_n = i)
$$

Let $Q$ be the matrix that holds the probabilities of moving from transient states to transient states, let $\mu$ be the initial distribution.
The transient state distribution at step $n$ is

$$
\mu^{(n)} = \mu Q^n
$$

Hence, the total probability of not being absorbed by step $n$ is

$$
P(\text{not absorbed after n steps}) = P(\text{being in Q after n steps}) = 1^⊺ \mu Q^n
$$

In [4]:
# parameters
n_seconds = 300
n_rounds  = 3

In [5]:
fight = Fight(G=G, n_rounds=n_rounds, t=n_seconds)

In [6]:
Q = fight.Q
Q_n = np.linalg.matrix_power(Q, n_seconds)
p_in_Q = np.sum(Q_n[:, 0], axis=1)
p_in_Q_after_n_rounds = p_in_Q ** n_rounds

The probability of being in $Q$ can also be calculated using the first-hitting probabilities $h$

$$
P(\text{in Q after n steps}) = 1 - P(\text{hitting of s in within n steps})  = 1  -\sum_{i \in s}\sum_{x=0}^n h_s(x)
$$

In [7]:
j = [0, 1, 2, 3]                                                        # absorbing states
j = np.atleast_1d(j)
n_j = len(j)

In [8]:
n = n_seconds * n_rounds
n_samples = fight.n_samples

H     = np.zeros((n_samples, n_j, n))
mu_0  = np.repeat(fight.mu[:, None, :], n_j, axis=1)
P     = np.repeat(fight.P[:, None, :],  n_j, axis=1)

In [9]:
# "make absorbing"-trick
P[np.arange(n_samples)[:, None], np.arange(n_j), j, :] = 0
P[np.arange(n_samples)[:, None], np.arange(n_j), j, j] = 1

p_prev = np.zeros((n_samples, n_j))
p_Q    = np.ones(n_samples)

In [10]:
for i in range(n):
    if i == 0 or i % n_seconds == 0:                                  # start a round
        mu_n = mu_0 * p_Q[:, None, None]                              # put remaining probability mass into mu

    mu_n = np.einsum('sjn, sjnk -> sjk', mu_n, P)
    p = mu_n[np.arange(n_samples)[:, None], np.arange(n_j), j]
    h = np.maximum(p - p_prev, 0.0)
    H[:, :, i] = h                                                    # enforce non-negativity
    p_prev = p
    p_Q -= np.sum(h, axis=1)

In [11]:
p_absorbed = np.sum(H, axis=(1, 2))
p_not_absorbed_after_n_rounds = 1 - p_absorbed

In [12]:
# p_transient
Q = fight.Q
Q_n = np.linalg.matrix_power(Q, n_seconds)
p_in_Q = np.sum(Q_n[:, 0], axis=1)
p_in_Q_after_n_rounds = p_in_Q ** n_rounds

In [13]:
bool(np.allclose(p_in_Q_after_n_rounds, p_not_absorbed_after_n_rounds))

True